In [1]:
import pandas as pd
import numpy as np
import datetime as dt
import json
from datetime import datetime

import plotly.io as pio
import plotly.graph_objects as go
import plotly.express as px
import warnings
warnings.filterwarnings('ignore')


# this enables relative path imports
import os
from dotenv import load_dotenv
load_dotenv()
_PROJECT_PATH: str = os.environ["_project_path"]
_PICKLED_DATA_FILENAME: str = os.environ["_pickled_data_filename"]

import sys
from pathlib import Path
project_path = Path(_PROJECT_PATH)
sys.path.append(str(project_path))

import config_v2 as cfg

from library_report_v2 import Cleaning as cln
from library_report_v2 import Graphing as grp
from library_report_v2 import Processing as pro
from library_report_v2 import Configuration as repcfg

Baseline en config_v2.py: ['2023-01-01', '2025-03-10']
Study en config_v2.py: ['2025-03-10', '2025-03-17']


In [2]:
def show_response_contents(df):
    print("The response contains:")
    print(json.dumps(list(df['variable'].unique()), sort_keys=True, indent=4))
    print(json.dumps(list(df['device'].unique()), sort_keys=True, indent=4))

In [3]:
df = pd.read_pickle(project_path / 'data' / _PICKLED_DATA_FILENAME)
#show_response_contents(df)

In [4]:
df = df.sort_values(by=['variable','datetime'])
df = pro.datetime_attributes(df)

#df_bl, df_st = pro.split_into_baseline_and_study(df, baseline=cfg.BASELINE, study=cfg.STUDY, inclusive='left')
#df_bl['Periodo'] = 'Baseline'
#df_st['Periodo'] = 'Estudio'

#past_w = df_bl.loc[cfg.PAST_WEEK[0]:cfg.PAST_WEEK[1]]

In [5]:
#Del dataframe completo (baselin-study) se saca solo la variable energía activa
front = df.query("variable == 'front-consumo-activa'").copy()

#Se agrupan los datos por intervalos de una hora
front_hour = front.groupby(by=["variable", "device_name"]).resample('1h').sum().round(2).reset_index().set_index('datetime')
front_hour = pro.datetime_attributes(front_hour)


# Horario

In [6]:
# Seleccionar columnas específicas directamente
front_hour = front_hour[['variable', 'device_name', 'value', 'hour', 'day', 'month', 'year', 'dow']]


In [7]:
front_hour

,variable,device_name,value,hour,day,month,year,dow
datetime,,,,,,,,
2024-02-26 18:00:00-05:00,front-consumo-activa,BC 687 - Planeta Rica,0.00,18,26,2,2024,lunes
2024-02-26 19:00:00-05:00,front-consumo-activa,BC 687 - Planeta Rica,0.00,19,26,2,2024,lunes
2024-02-26 20:00:00-05:00,front-consumo-activa,BC 687 - Planeta Rica,0.00,20,26,2,2024,lunes
2024-02-26 21:00:00-05:00,front-consumo-activa,BC 687 - Planeta Rica,0.00,21,26,2,2024,lunes
2024-02-26 22:00:00-05:00,front-consumo-activa,BC 687 - Planeta Rica,0.00,22,26,2,2024,lunes
...,...,...,...,...,...,...,...,...
2025-01-26 20:00:00-05:00,front-consumo-activa,BC 892 - La Vega,0.33,20,26,1,2025,domingo
2025-01-26 21:00:00-05:00,front-consumo-activa,BC 892 - La Vega,0.37,21,26,1,2025,domingo
2025-01-26 22:00:00-05:00,front-consumo-activa,BC 892 - La Vega,0.34,22,26,1,2025,domingo


In [8]:
#front_hour = front_hour[front_hour.index.year == 2024]

In [9]:
front_hour.to_clipboard(index=True, header=True)

# Diario

In [7]:
front_day = front.groupby(by=["variable", "device_name"]).resample('1D').sum().round(2).reset_index().set_index('datetime')
front_day = pro.datetime_attributes(front_day)

In [8]:
front_day = front_day[['variable', 'device_name', 'value', 'day', 'month', 'year', 'dow']]

In [9]:
front_day = front_day[front_day.index.year == 2025]

In [11]:
front_day.to_clipboard(index = True, header = True)

In [14]:
#front_hour = front.groupby(['device_name', 'hour']).agg({'value': 'mean', 'day': 'first', 'dow':'first',
#       'cont_dow':'first', 'week':'first', 'year':'first' }).reset_index().round(2)

In [10]:
front_day

,variable,device_name,value,day,month,year,dow
datetime,,,,,,,
2025-01-01 00:00:00-05:00,front-consumo-activa,BC 687 - Planeta Rica,88.00,1,1,2025,miércoles
2025-01-02 00:00:00-05:00,front-consumo-activa,BC 687 - Planeta Rica,186.00,2,1,2025,jueves
2025-01-03 00:00:00-05:00,front-consumo-activa,BC 687 - Planeta Rica,174.00,3,1,2025,viernes
2025-01-04 00:00:00-05:00,front-consumo-activa,BC 687 - Planeta Rica,78.00,4,1,2025,sábado
2025-01-05 00:00:00-05:00,front-consumo-activa,BC 687 - Planeta Rica,75.00,5,1,2025,domingo
...,...,...,...,...,...,...,...
2025-03-13 00:00:00-05:00,front-consumo-activa,BC 892 - La Vega,54.50,13,3,2025,jueves
2025-03-14 00:00:00-05:00,front-consumo-activa,BC 892 - La Vega,58.58,14,3,2025,viernes
2025-03-15 00:00:00-05:00,front-consumo-activa,BC 892 - La Vega,8.42,15,3,2025,sábado
